In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from copy import copy
from mpl_toolkits.mplot3d import axes3d
from scipy.interpolate import griddata, interp2d

from pathlib import Path

ncolors = 256
color_array = plt.get_cmap('terrain')(range(ncolors))
color_array[:,-1] = np.linspace(1.0,0.0,ncolors)
map_object = colors.LinearSegmentedColormap.from_list(name='terrain_alpha',colors=color_array)
plt.register_cmap(cmap=map_object)


In [ ]:
paths = []
results = {}

# # phi vs beta vs epsilon * l
# paths.append(Path("../../results/distances/20220821-212539.json"))

# big something
# paths.append(Path("../../results/distances/20220826-141540.json"))

# phi epsilon with memory
# paths.append(Path("../../results/distances/20220826-164153.json"))

# phi el outbound with memory
# paths.append(Path("../../results/distances/phi-el-outbound-record.json"))

# el outbound beta with memory
paths.append(Path("../../results/distances/el-outbound-beta-record.json"))

# motor beta outbound (no mem)
# paths.append(Path("../../results/distances/motor-beta-outbound.json"))

# phi beta epsilon
# paths.append(Path("../../results/distances/phi-beta-epsilon-non-cheat.json"))
# paths.append(Path("../../results/distances/phi-beta-epsilon-cheat.json"))

for path in paths:
    with path.open() as f:
        result = json.load(f)
        results.update(result)

In [ ]:
parameter = "T_outbound"
data = [(v["parameters"][parameter],v["mean_min_distance"]) for v in results.values()]
data = sorted(data)

# visited_data = set()

# out = []
# for a, b in data:
#     if not a in visited_data:
#         visited_data.add(a)
#         out.append((a, b))
        
# data = out

min_x,min_y = zip(min(data, key=lambda tup: tup[1]))
x,y = zip(*data)

plt.figure(figsize=(10,10))
plt.plot(x,y)
plt.scatter(x,y)
plt.plot(min_x, min_y, marker="o", markersize=10, color="g",
            label=f"minimum {min_y[0]} at {parameter}={min_x[0]}")
# plt.xscale("log")
plt.xlabel(parameter)
plt.ylabel("Mean closest distance")
plt.legend()


In [ ]:
# 3D plots
parameter_1 = "phi"
parameter_2 = "T_outbound"
data = [(v["parameters"]["cx"]["params"][parameter_1],
         v["parameters"][parameter_2],
         v["mean_min_distance"]/v["mean_homing_distance"]) 
        for v in results.values() if not v["parameters"]["cx"]["params"]["cheat"] and v["parameters"]["cx"]["params"]["phi"]]

data = sorted(data)
# data = [tup for tup in data if tup[1] > 1e-2]
# print(len(data))

minimum = zip(min(data, key=lambda tup: tup[2]))
print(*minimum)

x,y,z = zip(*data)
x = np.log10(x)
# y = np.log10(y)

grid_x,grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]
grid_z = griddata((x,y),z,(grid_x,grid_y),method='linear',rescale=True) # method can be nearest, linear or cubic

plt.figure(figsize=(9,9))

ax = plt.axes(projection='3d', computed_zorder=False)
ax.plot_surface(grid_x,grid_y,grid_z,cmap="terrain")
ax.view_init(elev=90, azim=0)

plt.xlabel(f"{parameter_1}")
plt.ylabel(f"{parameter_2}")

color_map = cm.ScalarMappable(cmap="terrain")
color_map.set_array(z)

plt.colorbar(color_map,fraction=0.03, pad=0.02)
plt.show()

# 3D plots
data = [(v["parameters"]["cx"]["params"][parameter_1],v["parameters"][parameter_2],v["mean_min_distance"]/v["mean_homing_distance"]) for v in results.values() if v["parameters"]["cx"]["params"]["cheat"]]

data = sorted(data)

minimum = zip(min(data, key=lambda tup: tup[2]))
print(*minimum)

x,y,z = zip(*data)
# x = np.log10(x)
# y = np.log10(y)

grid_x,grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]
grid_z = griddata((x,y),z,(grid_x,grid_y),method='linear',rescale=True) # method can be nearest, linear or cubic

plt.figure(figsize=(9,9))

ax = plt.axes(projection='3d', computed_zorder=False)
ax.plot_surface(grid_x,grid_y,grid_z,cmap="terrain")
ax.view_init(elev=90, azim=0)

plt.xlabel(f"{parameter_1}$")
plt.ylabel(f"{parameter_2}")

color_map = cm.ScalarMappable(cmap="terrain")
color_map.set_array(z)

plt.colorbar(color_map,fraction=0.03, pad=0.02)
plt.show()

In [ ]:
# 3D plots
parameter_1 = "motor_factor"
parameter_2 = "T_outbound"
data = [(v["parameters"][parameter_1]*10,v["parameters"][parameter_2],v["mean_min_distance"]/v["mean_homing_distance"]) for v in results.values() if v["parameters"]["cx"]["params"]["cheat"]]

data = sorted(data)
# data = [tup for tup in data if tup[0] > 1e-2]
# print(len(data))

minimum = zip(min(data, key=lambda tup: tup[2]))
print(*minimum)

x,y,z = zip(*data)
# x = np.log10(x)
# y = np.log10(y)
# z = np.log10(z)
grid_x,grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]
grid_z = griddata((x,y),z,(grid_x,grid_y),method='',rescale=False) # method can be nearest, linear or cubic

plt.figure(figsize=(9,9))

ax = plt.axes(projection='3d', computed_zorder=False)
ax.plot_surface(grid_x,grid_y,grid_z,cmap="terrain")
ax.view_init(elev=90, azim=0)
# ax.set_zlim(1e3,1e7)

plt.xlabel(f"{parameter_1}")
plt.ylabel(f"{parameter_2}")
# plt.rcParams['font.size'] = 11

color_map = cm.ScalarMappable(cmap="terrain")
color_map.set_array(z)
plt.colorbar(color_map,fraction=0.03, pad=0.02)
plt.show()

In [ ]:
### 4D plots ###
parameter_1 = "epsilon"
parameter_2 = "T_outbound"
parameter_3 = "beta"
data = [(v["parameters"]["cx"]["params"][parameter_1],v["parameters"][parameter_2],v["parameters"]["cx"]["params"][parameter_3],v["mean_memory_rmse"]) for v in results.values() if not v["parameters"]["cx"]["params"]["cheat"]]

data_copy = data
dist_lim = 2
data = [tup for tup in data if tup[3] <= dist_lim]
minimum = zip(min(data, key=lambda tup: tup[3]))
print(*minimum)

# Datapoints
xc,yc,zc,hc = zip(*data_copy)
x,y,z,h = zip(*data)
x = np.log10(x)
# y = np.log10(y)
# z = np.log10(z)

# Figure
fig = plt.figure(figsize=(9,9))
ax = plt.axes(projection='3d')
ax.view_init(elev=45, azim=45)

# Colormap
color_map = cm.ScalarMappable(cmap="terrain")
color_map.set_array(hc)
plt.colorbar(color_map, fraction=0.03, pad=0.04)

# Labels
ax.set_xlabel(f"{parameter_1}")
ax.set_ylabel(f"{parameter_2}")
ax.set_zlabel(f"{parameter_3}")

# Limits
# ax.set_xlim(-2.5,3.5)
# ax.set_ylim(-400,4500)
# ax.set_zlim(-0.01,0.51)

# Scatterplot data
ax.scatter(x,y,z,c=h,s=140,marker='h',cmap=color_map.get_cmap(), norm=colors.Normalize(min(hc),max((hc))))
# plt.scatter(x,y,z)
plt.show()